In [ ]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile
from sklearn.model_selection import StratifiedKFold

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train['has_anno'] = train.annotations != '[]'
train = train.loc[train['has_anno'] == True]
train.reset_index(drop=True, inplace=True)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(skf.split(train, train["video_id"])):
    train.loc[val_idx, 'fold'] = fold

In [ ]:
HOME_DIR = '/kaggle/working'
!mkdir -p ./yolov5_data/fold3/images/val
!mkdir -p ./yolov5_data/fold3/images/train

!mkdir -p ./yolov5_data/fold3/labels/val
!mkdir -p ./yolov5_data/fold3/labels/train

In [ ]:
fold = 3

annos = []
for i, x in train.iterrows():
    if x['fold'] == fold:
        mode = 'val'
    else:
        mode = 'train'

    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolov5_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    r = ''
    anno = eval(x.annotations)
    for an in anno:
        w = an['width'] 
        h = an['height']
    
        if (an['x'] + an['width'] > 1280):
            w = 1280 - an['x'] 
        if (an['y'] + an['height'] > 720):
            h = 720 - an['y'] 
        
        r += '0 {} {} {} {}\n'.format((an['x'] + int(np.round(w/2))) / 1280,
                                        (an['y'] + int(np.round(h/2))) / 720,
                                        w / 1280, h / 720)
    with open(f'./yolov5_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [ ]:
train_path = '/kaggle/working/yolov5_data/fold3/images/train'
val_path = '/kaggle/working/yolov5_data/fold3/images/val'
cwd  = '/kaggle/working'
images_train = os.listdir(train_path)
images_val = os.listdir(val_path)
with open(os.path.join(cwd , 'train.txt'), 'w') as f:
    for path in images_train:
        f.write(train_path+'/'+path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in images_val:
        f.write(val_path+'/'+path+'\n')

In [ ]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5
!git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

In [ ]:
hyps = '''
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [ ]:
data = '''
train: /kaggle/working/train.txt  # train images (relative to 'path')
val: /kaggle/working/val.txt  # val images (relative to 'path')
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['starfish']  # class names
'''

In [ ]:
with open(f'{HOME_DIR}/Yolov5-protect-reef.yaml', 'w') as fp:
    fp.write(data)
with open(f'{HOME_DIR}/hyp-yolov5.yaml', 'w') as fp:
    fp.write(hyps)

In [ ]:
!wandb off

In [ ]:
!python train.py --img 3500\
--batch 4\
--epochs 7\
--optimizer 'Adam'\
--data '{HOME_DIR}/Yolov5-protect-reef.yaml'\
--hyp '{HOME_DIR}/hyp-yolov5.yaml'\
--weights 'yolov5n6.pt'\
--project 'Protect_reef' --name 'yolov5n6'\
--exist-ok